# TrusstGPT Model Package from AWS Marketplace 


The TrustGPT model is designed for post-call analytics on customer call transcripts. It takes a call transcript as the input, and extracts 14 different features from the call. These features are output as a structured JSON file. The format of the input is as follows:

    """<s> ###Instruction:
    Extract summary and key information from the input below in valid json format.

    ### Input:
    A: Thanks for calling how can I help you? 
    C: Can you please do my taxes? 
    A: Sure, all done. 
    C: Thanks.
    
    ### JSON Output:"""

and then the model will output the following:

    {'SUMMARY': 'Can you please do my taxes?',
     'INTENT': 'Taxes',
     'RESOLUTION': 'Yes',
     'PRODUCT': 'Taxes',
     'PROBLEM_SIGNATURE': 'Can you please do my taxes?',
     'AGENT_ACTION': 'The agent did the taxes.',
     'AGENT_MANNER': 'Polite',
     'CUSTOMER_MANNER': 'Polite',
     'REPEAT_CALL': 'No',
     'CATEGORY_1': 'Tax',
     'CATEGORY_2': 'Tax',
     'CATEGORY_3': 'Tax preparation'}

This sample notebook shows you how to deploy [TrusstGPT](https://aws.amazon.com/marketplace/pp/prodview-oaa3fdh5wsx4y) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to <font color='red'> For Seller to update:[Title_of_your_ML Model](Provide link to your marketplace listing of your product)</font>. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [TrusstGPT](https://aws.amazon.com/marketplace/pp/prodview-oaa3fdh5wsx4y).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Note that the product is only currently available in ap-southeast-2 region. Please contact contact@trusst.ai if you require a different region.
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn = "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/trusstgpt-73a8994b20503c7883c1916e078274cd"

<font color='red'> For Seller to update: Add all necessary imports in following cell, 
If you need specific packages to be installed, # try to provide them in this section, in a separate cell. </font>

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np
from sagemaker.huggingface import HuggingFacePredictor

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

'sagemaker-ap-southeast-2-967820297265'

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "TrusstGPT"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.12xlarge"
)
batch_transform_inference_instance_type = (
    "ml.m5.large"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------------

Once endpoint has been created, you would be able to perform real-time inference.

### C. Perform real-time inference

In [ ]:
text = """<s> ###Instruction:
Extract summary and key information from the input below in valid json format.

### Input:
A: Thanks for calling how can I help you? C: Can you please do my taxes? A: Sure, all done. C: Thanks.

### JSON Output:"""

parameters = {
    "max_new_tokens": 500,
    "max_length": 4096,
    "do_sample": False,
    "temperature": 0.2,
    "top_p": 0.2,
    "pad_token_id": 50256,
    "use_cache":True
}

llm = HuggingFacePredictor(endpoint_name=model.model_name)
payload = {"inputs": text, "parameters": parameters}
response = llm.predict(payload)

Please note that only real-time endpoints are supported at this time as the inference is based on the TGI container which is optimised for real-time inference.

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

<font color='red'>For Seller to update: review/update file_name, output_file name, custom attributes in following AWS CLI to perform a real-time inference using the payload file you created from 2.B </font>

In [ ]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

### D. Visualize output

In [ ]:
json.loads(response[0]["generated_text"])

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

